In [3]:
# Libraries
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import numpy as np

In [2]:
# Helper Methods
def euclidean_dist(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

In [4]:
# Data preparation
X, y = make_blobs(
    centers=3, n_samples=1000, n_features=2, shuffle=True, random_state=42
)
print(X.shape)
n_clusters = len(np.unique(y))
print(n_clusters)

(1000, 2)
3


In [5]:
class KMeans:
    def __init__(self, n_clusters=5, max_iters=100, plot_steps=False):
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        self.plot_steps = plot_steps

        self.clusters = [[] for _ in range(self.n_clusters)]
        self.centroids = []

    def _create_clusters(self, centroids):
        pass

    def _get_cluster_labels(self, clusters):
        pass

    def _get_centroids(self, clusters):
        pass

    def _closest_centroid(self, sample, centroids):
        pass

    def _is_converged(self, old_centroids, centroids):
        pass
    
    def predict(self, X):
        pass
    
    def plot(self):
        pass